In [1]:
import numpy as np
import pandas as pb
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import os
from tensorflow.keras.models import load_model




Setting path for dataset

In [2]:
test_data_dir = 'D:\\python\\chest_xray\\test'
train_data_dir = 'D:\\python\\chest_xray\\train'
validation_data_dir = 'D:\\python\\chest_xray\\val'



Check the data directory structure

In [3]:


# Directories
test_data_dir = 'D:\\python\\chest_xray\\test'
train_data_dir = 'D:\\python\\chest_xray\\train'
validation_data_dir = 'D:\\python\\chest_xray\\val'

# Define batch size and image dimensions
batch_size = 32
img_height = 150
img_width = 150

# Train Data Generator: Augmentations for training
train_datagen = ImageDataGenerator(
    rescale=1./255,           # Normalize pixel values to [0, 1]
    shear_range=0.3,          # Apply random shear transformations
    zoom_range=0.4,           # Apply random zoom
    horizontal_flip=True      # Randomly flip images horizontally
)

# Validation Data Generator: No augmentation, only rescaling
validation_datagen = ImageDataGenerator(rescale=1./255)

# Test Data Generator: No augmentation, only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# Train data
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),  # Resize images to target size
    batch_size=batch_size,
    class_mode='categorical'  # Assuming you have multiple classes
)

# Validation data
validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Test data
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',  # Assuming multi-class classification
    shuffle=False  # No need to shuffle test data
)


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


varifying loaded data

In [4]:
# Print class indices for verification
print("Class indices: ", train_generator.class_indices)


Class indices:  {'NORMAL': 0, 'PNEUMONIA': 1}


BUILDING A SIMPLE CNN MODEL

In [5]:
model= Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150,150, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

#compile model

model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Since you have multiple classes
              metrics=['accuracy'])

C:\Users\Adil\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
#Train the model

history = model.fit(
    train_generator,
    epochs=5,  # Adjust based on needs
    validation_data=validation_generator
)



Epoch 1/5


C:\Users\Adil\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


163/163 ━━━━━━━━━━━━━━━━━━━━ 224s 1s/step - accuracy: 0.7168 - loss: 0.6882 - val_accuracy: 0.6250 - val_loss: 0.9740
Epoch 2/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 125s 748ms/step - accuracy: 0.8392 - loss: 0.3724 - val_accuracy: 0.7500 - val_loss: 0.6731
Epoch 3/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 125s 746ms/step - accuracy: 0.8761 - loss: 0.2840 - val_accuracy: 0.6250 - val_loss: 1.0912
Epoch 4/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 126s 752ms/step - accuracy: 0.9035 - loss: 0.2316 - val_accuracy: 0.6250 - val_loss: 0.9360
Epoch 5/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 127s 758ms/step - accuracy: 0.9109 - loss: 0.2318 - val_accuracy: 0.8125 - val_loss: 0.4293


In [7]:
#Evaluate the model on test data

test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")


20/20 ━━━━━━━━━━━━━━━━━━━━ 16s 782ms/step - accuracy: 0.8854 - loss: 0.3549
Test Accuracy: 0.9086538553237915


In [8]:
#summary the model

# Print a summary of the model
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    18,940,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │         1,026 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 57,104,072 (217.83 MB)

 Trainable params: 19,034,690 (72.61 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 38,069,382 (145.22 MB)

In [9]:
#Save the model

model.save('chest_xray_classification_model.keras')
